In [1]:
user_request = input("Hey, how can I help you today?")

print(user_request)

i want to make a todo list app named JustDoIt


In [23]:
%pip install -q langchain-google-genai
from langchain_google_genai import ChatGoogleGenerativeAI

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
%pip install python-dotenv
from dotenv import load_dotenv

load_dotenv()

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


True

In [25]:
# To use Groq
# llm = ChatGroq(
#     model="llama3-70b-8192",
#     temperature=0.0,
#     max_retries=2,
#     verbose=True
# )

# To use Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


messages = [
    ("system", "You are a senior engineer working for a professional software company. Your task is to convert ideas into actionable steps. You will be prompted with a general ideas and you need to respond back with how to make it with proper steps instructions."),
    ("human", user_request),
]

In [26]:
print(llm.invoke(messages))

# To stream the message
#
# for chunk in llm.stream(messages):
#     print(chunk.content, end="")

content='Let\'s break down building a "JustDoIt" to-do list app into actionable steps.  We\'ll consider a basic version initially, focusing on core functionality, and then discuss potential expansions.\n\n**Phase 1: Planning and Design (1-2 days)**\n\n1. **Define Core Features:**\n    * **Task Creation:**  Add new tasks with a description and optional due date/time.\n    * **Task Listing:** Display all tasks, possibly categorized (e.g., by due date, priority).\n    * **Task Completion:** Mark tasks as complete.\n    * **Task Deletion:** Remove completed or unwanted tasks.\n    * **Persistence:** Save and load tasks persistently (so data isn\'t lost on app closure).\n\n2. **Choose a Technology Stack:**\n    * **Frontend:**  React, Vue, Angular, or even a simpler framework like Svelte or vanilla JavaScript.  Consider the team\'s expertise and project timeline.  React is a popular and robust choice.\n    * **Backend (Optional):** For a simple app, local storage might suffice.  For more ad

In [7]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
import json

In [8]:
# Define the expected JSON structure
parser = JsonOutputParser(pydantic_object={
    "type": "object",
    "properties": {
        "steps": {
            "type": "array",
            "items": {
                "command": {"type": "string"},
                "description": {
                    "type": "string",
                }
            }
        }
    }
})

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """Create a step by step instruction to follow with terminal commands to run on windows to setup a project based on user needs. Understand the user needs if its a frontend project use vite react, if its backend use express
     These commands will be actually be implemented by an autonomous agent so make sure each and every minute details is included and you dont make any mistake:
     
    {{
        "steps": [{{
            "command": "mkdir frontend",
            "description": "Initializing directory"
        }},...]
    }}
     Do not include the actual coding implementation. Do not write any code in any file.
     reply with just json and nothing else.
     """),
    ("user", "User is requesting: {input} {feedback_prompt}")
])

In [30]:
steps_generation_chain = prompt | llm | parser

result = steps_generation_chain.invoke({"input": user_request, "feedback_prompt": None})
print(json.dumps(result, indent=2))

{
  "steps": [
    {
      "command": "mkdir JustDoIt",
      "description": "Create the main project directory"
    },
    {
      "command": "cd JustDoIt",
      "description": "Navigate into the project directory"
    },
    {
      "command": "mkdir frontend",
      "description": "Create a directory for the frontend"
    },
    {
      "command": "cd frontend",
      "description": "Navigate into the frontend directory"
    },
    {
      "command": "npm init vite@latest JustDoIt -- --template react",
      "description": "Initialize a Vite React project named JustDoIt.  This will take some time to download dependencies."
    },
    {
      "command": "cd ..",
      "description": "Navigate back to the main project directory"
    },
    {
      "command": "mkdir backend",
      "description": "Create a directory for the backend"
    },
    {
      "command": "cd backend",
      "description": "Navigate into the backend directory"
    },
    {
      "command": "npm init -y",
      

In [32]:
messages = [
    ("system", "You are a senior engineer who is an expert at finding mistakes. Your task is to find mistakes from a set of actions and give feedback. If all steps are good enough reply back with just one word approved else give feedback."),
    ("human", "here are my step by step idea to make a to do list app named JustDoIt" + json.dumps(result)),
]

response = llm.invoke(messages)
feedback = response.content
print(feedback)

The project initialization is good, but there's a crucial naming conflict. You create a project directory and a frontend project with the same name, "JustDoIt".  This will lead to confusion and potential problems later.  The frontend project should have a distinct name (e.g., `justdoit-frontend`).  Rename the frontend project or change the project name in the `npm init` command to avoid this conflict.


In [34]:
feedback_prompt = None
if feedback:
    feedback_prompt = f"In the previous response, you made a few mistake here: {result}. I have a few feedback. Consider this and regenerate: {feedback}"

steps_generation_chain.invoke({"input": user_request, "feedback_prompt": feedback_prompt})

{'steps': [{'command': 'mkdir JustDoIt',
   'description': 'Create the main project directory'},
  {'command': 'cd JustDoIt',
   'description': 'Navigate into the project directory'},
  {'command': 'mkdir frontend',
   'description': 'Create a directory for the frontend'},
  {'command': 'cd frontend',
   'description': 'Navigate into the frontend directory'},
  {'command': 'npm init vite@latest JustDoIt -- --template react',
   'description': 'Initialize a Vite React project named JustDoIt.  This will take some time to download dependencies.'},
  {'command': 'cd ..',
   'description': 'Navigate back to the main project directory'},
  {'command': 'mkdir backend',
   'description': 'Create a directory for the backend'},
  {'command': 'cd backend',
   'description': 'Navigate into the backend directory'},
  {'command': 'npm init -y',
   'description': 'Initialize a Node.js project.  -y automatically accepts default values.'},
  {'command': 'npm install express',
   'description': 'Install

In [35]:
print(feedback)

The project initialization is good, but there's a crucial naming conflict. You create a project directory and a frontend project with the same name, "JustDoIt".  This will lead to confusion and potential problems later.  The frontend project should have a distinct name (e.g., `justdoit-frontend`).  Rename the frontend project or change the project name in the `npm init` command to avoid this conflict.
